# PyTorch数据并行处理

In [ ]:
import torch
import torch.nn as nn

In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [17]:
# 哪些东西 需要给device
model.to(device)
data.to(device)#tensor

device(type='cuda', index=0)

In [ ]:
# 并行
model = nn.DataParallel(model)

In [19]:
# toy data & model
from torch.utils.data import Dataset,DataLoader
# 超参
input_size = 5
output_size = 2
batch_size = 30
data_size = 100


In [21]:
# 新数据 包装成 DataLoader
class RandomDataset(Dataset):
    def __init__(self,input_size,data_size):
        self.len = data_size
        self.data = torch.randn(data_size,input_size)
    def __getitem__(self,index):
        return self.data[index]
    def __len__(self):
        return self.len
rand_loader = DataLoader(dataset = RandomDataset(input_size,data_size),batch_size=batch_size,shuffle=True)

In [24]:
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        super(Model,self).__init__()
        self.fc = nn.Linear(input_size,output_size)
    def forward(self,input):
        output = self.fc(input)
        print("\tIn Model: input size",input.size(),
             "Output size",output.size())
        return output
model = Model(input_size,output_size)
print("Let us use",torch.cuda.device_count(),"GPUS!")
model = nn.DataParallel(model)
model.to(device)

Let us use 4 GPUS!


DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

In [25]:
# train
for data in rand_loader:
    input = data.to(device)
    output = model(input)
    print("Outside:input size",input.size(),"output_size",output.size())

	In Model: input size	In Model: input size 	In Model: input sizetorch.Size([8, 5])  torch.Size([8, 5]) Output size torch.Size([8, 5])	In Model: input size Output size  torch.Size([8, 2]) Output sizetorch.Size([6, 5])
torch.Size([8, 2])  Output size
torch.Size([8, 2]) Outside:input size
torch.Size([6, 2]) 
torch.Size([30, 5]) output_size torch.Size([30, 2])
	In Model: input size 	In Model: input size	In Model: input size	In Model: input sizetorch.Size([8, 5])    torch.Size([6, 5])torch.Size([8, 5])torch.Size([8, 5])Output size    Output sizeOutput sizeOutput sizetorch.Size([8, 2])   Outside:input size
torch.Size([6, 2])torch.Size([8, 2])torch.Size([8, 2]) torch.Size([30, 5])


 output_size torch.Size([30, 2])
	In Model: input size torch.Size([8, 5]) 	In Model: input size	In Model: input size	In Model: input sizeOutput size    torch.Size([8, 5])torch.Size([8, 5])torch.Size([6, 5])torch.Size([8, 2])   Outside:input size
Output sizeOutput sizeOutput size torch.Size([30, 5])    torch.Size([